In [1]:
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

import os
import shutil
import time
import math

from PIL import Image
import numpy as np
import sys

import torch
from torch import nn
from torch.autograd import Variable, Function
from torchvision import models, transforms
import torchvision.models as models
import torchvision.datasets as datasets
import torch.utils.model_zoo as model_zoo

%matplotlib inline

print("mpl backend: ", plt.get_backend())

/home/lestsovba/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


mpl backend:  module://ipykernel.pylab.backend_inline


In [2]:
torch.cuda.set_device(3)

In [113]:
traindir = '/home/lestsovba/dataset/imagenet/registered/tiny-imagenet-200/train/'
valdir = '/home/lestsovba/dataset/imagenet/registered/tiny-imagenet-200/val/'
workers = 1
batch_size = 16

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    root=traindir,
    transform=transforms.Compose([
        transforms.CenterCrop(64),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

val_dataset = datasets.ImageFolder(
    root=valdir,
    transform=transforms.Compose([
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size,
    shuffle=True,
    num_workers=workers, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size, shuffle=False,
    num_workers=workers, pin_memory=True)


In [129]:
resume='checkpoint.pth.tar'
if os.path.isfile(resume):
    print("=> loading checkpoint '{}'".format(resume))
    checkpoint = torch.load(resume)
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})"
          .format(resume, checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(resume))

=> loading checkpoint 'checkpoint.pth.tar'
=> loaded checkpoint 'checkpoint.pth.tar' (epoch 1)


In [25]:
model = vgg11_bn_little(num_classes=200)
model = model.cuda()

In [114]:
criterion = nn.BCEWithLogitsLoss().cuda()

init_lr = 0.1
optimizer = torch.optim.SGD(model.parameters(), init_lr,
                            momentum=0.9,
                            weight_decay=1e-4)

In [117]:
print_freq = 10
epochs = 10
for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    prec1 = validate(val_loader, model, criterion)

    # remember best prec@1 and save checkpoint
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': 'vgg11_bn_little',
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)


Epoch: [0][0/6250]	Time 0.203 (0.203)	Data 0.088 (0.088)	Loss 0.0338 (0.0338)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][10/6250]	Time 0.181 (0.182)	Data 0.000 (0.008)	Loss 0.0328 (0.0332)	Prec@1 0.000 (0.568)	Prec@5 0.000 (2.273)
Epoch: [0][20/6250]	Time 0.180 (0.181)	Data 0.000 (0.004)	Loss 0.0334 (0.0335)	Prec@1 0.000 (0.298)	Prec@5 0.000 (1.786)
Epoch: [0][30/6250]	Time 0.181 (0.181)	Data 0.000 (0.003)	Loss 0.0352 (0.0336)	Prec@1 0.000 (0.403)	Prec@5 0.000 (2.621)
Epoch: [0][40/6250]	Time 0.180 (0.181)	Data 0.000 (0.002)	Loss 0.0341 (0.0336)	Prec@1 0.000 (0.305)	Prec@5 6.250 (2.287)
Epoch: [0][50/6250]	Time 0.180 (0.181)	Data 0.000 (0.002)	Loss 0.0324 (0.0335)	Prec@1 0.000 (0.368)	Prec@5 0.000 (2.206)
Epoch: [0][60/6250]	Time 0.180 (0.181)	Data 0.000 (0.002)	Loss 0.0328 (0.0335)	Prec@1 0.000 (0.410)	Prec@5 0.000 (2.152)
Epoch: [0][70/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0324 (0.0335)	Prec@1 6.250 (0.440)	Prec@5 6.250 (2.113)
Epoch: [0][80/6250]	Time 0.180 (0

Epoch: [0][680/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0322 (0.0328)	Prec@1 0.000 (0.597)	Prec@5 6.250 (3.019)
Epoch: [0][690/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0321 (0.0328)	Prec@1 0.000 (0.588)	Prec@5 0.000 (3.030)
Epoch: [0][700/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0336 (0.0328)	Prec@1 0.000 (0.588)	Prec@5 0.000 (3.049)
Epoch: [0][710/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0312 (0.0327)	Prec@1 0.000 (0.598)	Prec@5 18.750 (3.094)
Epoch: [0][720/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0317 (0.0327)	Prec@1 0.000 (0.633)	Prec@5 0.000 (3.129)
Epoch: [0][730/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0327 (0.0327)	Prec@1 0.000 (0.624)	Prec@5 0.000 (3.121)
Epoch: [0][740/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0317 (0.0327)	Prec@1 0.000 (0.633)	Prec@5 0.000 (3.146)
Epoch: [0][750/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0324 (0.0327)	Prec@1 0.000 (0.657)	Prec@5 0.000 (3.171)
Epoch: [0][760/6250]	Ti

Epoch: [0][1350/6250]	Time 0.181 (0.181)	Data 0.000 (0.001)	Loss 0.0317 (0.0322)	Prec@1 0.000 (0.911)	Prec@5 0.000 (4.140)
Epoch: [0][1360/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0311 (0.0322)	Prec@1 0.000 (0.928)	Prec@5 6.250 (4.179)
Epoch: [0][1370/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0326 (0.0322)	Prec@1 0.000 (0.930)	Prec@5 0.000 (4.171)
Epoch: [0][1380/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0313 (0.0322)	Prec@1 0.000 (0.937)	Prec@5 6.250 (4.195)
Epoch: [0][1390/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0323 (0.0322)	Prec@1 0.000 (0.944)	Prec@5 0.000 (4.219)
Epoch: [0][1400/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0304 (0.0322)	Prec@1 0.000 (0.950)	Prec@5 6.250 (4.234)
Epoch: [0][1410/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0318 (0.0321)	Prec@1 0.000 (0.948)	Prec@5 6.250 (4.226)
Epoch: [0][1420/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0323 (0.0321)	Prec@1 6.250 (0.963)	Prec@5 6.250 (4.262)
Epoch: [0][1430/

Epoch: [0][2020/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0285 (0.0317)	Prec@1 0.000 (1.215)	Prec@5 25.000 (5.369)
Epoch: [0][2030/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0329 (0.0317)	Prec@1 0.000 (1.209)	Prec@5 0.000 (5.367)
Epoch: [0][2040/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0291 (0.0317)	Prec@1 0.000 (1.213)	Prec@5 25.000 (5.396)
Epoch: [0][2050/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0298 (0.0317)	Prec@1 6.250 (1.213)	Prec@5 6.250 (5.415)
Epoch: [0][2060/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0284 (0.0317)	Prec@1 6.250 (1.216)	Prec@5 18.750 (5.437)
Epoch: [0][2070/6250]	Time 0.181 (0.181)	Data 0.000 (0.001)	Loss 0.0312 (0.0317)	Prec@1 0.000 (1.219)	Prec@5 0.000 (5.450)
Epoch: [0][2080/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0307 (0.0317)	Prec@1 0.000 (1.213)	Prec@5 0.000 (5.439)
Epoch: [0][2090/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0304 (0.0317)	Prec@1 0.000 (1.220)	Prec@5 6.250 (5.467)
Epoch: [0][21

Epoch: [0][2690/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0281 (0.0313)	Prec@1 6.250 (1.489)	Prec@5 12.500 (6.457)
Epoch: [0][2700/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0294 (0.0313)	Prec@1 6.250 (1.493)	Prec@5 12.500 (6.477)
Epoch: [0][2710/6250]	Time 0.179 (0.181)	Data 0.000 (0.001)	Loss 0.0311 (0.0313)	Prec@1 0.000 (1.492)	Prec@5 0.000 (6.487)
Epoch: [0][2720/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0279 (0.0313)	Prec@1 6.250 (1.500)	Prec@5 25.000 (6.512)
Epoch: [0][2730/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0294 (0.0313)	Prec@1 0.000 (1.501)	Prec@5 12.500 (6.522)
Epoch: [0][2740/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0283 (0.0313)	Prec@1 6.250 (1.512)	Prec@5 18.750 (6.544)
Epoch: [0][2750/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0308 (0.0313)	Prec@1 0.000 (1.515)	Prec@5 12.500 (6.568)
Epoch: [0][2760/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0302 (0.0313)	Prec@1 0.000 (1.519)	Prec@5 12.500 (6.596)
Epoch: [0

...

Epoch: [0][4030/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0284 (0.0307)	Prec@1 6.250 (2.042)	Prec@5 12.500 (8.387)
Epoch: [0][4040/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0290 (0.0307)	Prec@1 12.500 (2.057)	Prec@5 18.750 (8.411)
Epoch: [0][4050/6250]	Time 0.181 (0.181)	Data 0.000 (0.001)	Loss 0.0321 (0.0306)	Prec@1 0.000 (2.060)	Prec@5 0.000 (8.419)
Epoch: [0][4060/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0282 (0.0306)	Prec@1 6.250 (2.064)	Prec@5 25.000 (8.439)
Epoch: [0][4070/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0312 (0.0306)	Prec@1 0.000 (2.065)	Prec@5 6.250 (8.442)
Epoch: [0][4080/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0277 (0.0306)	Prec@1 6.250 (2.075)	Prec@5 18.750 (8.460)
Epoch: [0][4090/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0313 (0.0306)	Prec@1 0.000 (2.076)	Prec@5 6.250 (8.462)
Epoch: [0][4100/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0303 (0.0306)	Prec@1 0.000 (2.080)	Prec@5 0.000 (8.470)
Epoch: [0][

Epoch: [0][4700/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0292 (0.0304)	Prec@1 6.250 (2.347)	Prec@5 6.250 (9.248)
Epoch: [0][4710/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0286 (0.0304)	Prec@1 0.000 (2.346)	Prec@5 6.250 (9.254)
Epoch: [0][4720/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0302 (0.0304)	Prec@1 0.000 (2.353)	Prec@5 18.750 (9.275)
Epoch: [0][4730/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0275 (0.0304)	Prec@1 12.500 (2.361)	Prec@5 18.750 (9.277)
Epoch: [0][4740/6250]	Time 0.181 (0.181)	Data 0.000 (0.001)	Loss 0.0287 (0.0304)	Prec@1 6.250 (2.364)	Prec@5 6.250 (9.286)
Epoch: [0][4750/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0276 (0.0304)	Prec@1 18.750 (2.373)	Prec@5 37.500 (9.303)
Epoch: [0][4760/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0289 (0.0304)	Prec@1 0.000 (2.376)	Prec@5 0.000 (9.305)
Epoch: [0][4770/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0308 (0.0304)	Prec@1 0.000 (2.384)	Prec@5 6.250 (9.323)
Epoch: [0][

Epoch: [0][5370/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0300 (0.0301)	Prec@1 0.000 (2.590)	Prec@5 6.250 (10.024)
Epoch: [0][5380/6250]	Time 0.181 (0.181)	Data 0.000 (0.001)	Loss 0.0303 (0.0301)	Prec@1 0.000 (2.595)	Prec@5 6.250 (10.034)
Epoch: [0][5390/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0281 (0.0301)	Prec@1 25.000 (2.604)	Prec@5 25.000 (10.049)
Epoch: [0][5400/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0297 (0.0301)	Prec@1 0.000 (2.611)	Prec@5 6.250 (10.068)
Epoch: [0][5410/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0270 (0.0301)	Prec@1 6.250 (2.610)	Prec@5 25.000 (10.073)
Epoch: [0][5420/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0302 (0.0301)	Prec@1 0.000 (2.614)	Prec@5 6.250 (10.071)
Epoch: [0][5430/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0253 (0.0301)	Prec@1 6.250 (2.613)	Prec@5 31.250 (10.081)
Epoch: [0][5440/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0308 (0.0301)	Prec@1 0.000 (2.612)	Prec@5 0.000 (10.082)
Epoc

Epoch: [0][6030/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0274 (0.0299)	Prec@1 12.500 (2.801)	Prec@5 25.000 (10.712)
Epoch: [0][6040/6250]	Time 0.179 (0.181)	Data 0.000 (0.001)	Loss 0.0300 (0.0299)	Prec@1 0.000 (2.806)	Prec@5 18.750 (10.727)
Epoch: [0][6050/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0286 (0.0299)	Prec@1 0.000 (2.807)	Prec@5 18.750 (10.734)
Epoch: [0][6060/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0291 (0.0299)	Prec@1 0.000 (2.811)	Prec@5 6.250 (10.744)
Epoch: [0][6070/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0279 (0.0299)	Prec@1 6.250 (2.810)	Prec@5 18.750 (10.756)
Epoch: [0][6080/6250]	Time 0.181 (0.181)	Data 0.000 (0.001)	Loss 0.0285 (0.0299)	Prec@1 6.250 (2.816)	Prec@5 12.500 (10.762)
Epoch: [0][6090/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0272 (0.0299)	Prec@1 0.000 (2.820)	Prec@5 12.500 (10.771)
Epoch: [0][6100/6250]	Time 0.180 (0.181)	Data 0.000 (0.001)	Loss 0.0273 (0.0299)	Prec@1 6.250 (2.821)	Prec@5 12.500 (10.779)


IndexError: index 200 is out of range for dimension 0 (of size 200)

In [130]:
best_prec1 = validate(val_loader, model, criterion)

Test: [0/625]	Time 0.188 (0.188)	Loss 0.0257 (0.0257)	Prec@1 0.000 (0.000)	Prec@5 6.250 (6.250)
Test: [10/625]	Time 0.098 (0.106)	Loss 0.0217 (0.0249)	Prec@1 0.000 (4.545)	Prec@5 0.000 (24.432)
Test: [20/625]	Time 0.098 (0.102)	Loss 0.0301 (0.0259)	Prec@1 0.000 (2.381)	Prec@5 0.000 (12.798)
Test: [30/625]	Time 0.098 (0.101)	Loss 0.0296 (0.0269)	Prec@1 0.000 (1.613)	Prec@5 25.000 (10.282)
Test: [40/625]	Time 0.098 (0.100)	Loss 0.0241 (0.0270)	Prec@1 0.000 (1.220)	Prec@5 0.000 (9.604)
Test: [50/625]	Time 0.098 (0.100)	Loss 0.0264 (0.0262)	Prec@1 0.000 (0.980)	Prec@5 0.000 (9.191)


Process Process-51:
Traceback (most recent call last):
  File "/home/lestsovba/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/lestsovba/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lestsovba/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/home/lestsovba/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/lestsovba/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/lestsovba/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/lestsovba/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardI

Test: [60/625]	Time 0.098 (0.100)	Loss 0.0256 (0.0264)	Prec@1 0.000 (0.820)	Prec@5 0.000 (7.684)


KeyboardInterrupt: 

In [127]:
save_checkpoint({
        'epoch': 1,
        'arch': 'vgg11_bn_little',
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer' : optimizer.state_dict(),
    }, False)


In [116]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target_cls) in enumerate(train_loader):
        # measure data loading time

        target = torch.zeros(batch_size, 200)
        for ind, cls in enumerate(target_cls):
            target[ind, cls] = 1
        target = target.float()
        target = target.cuda(async=True)
        
        input = input.float()
        input = input.cuda(async=True)
        
        data_time.update(time.time() - end)
        
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, torch.LongTensor(target_cls).cuda(), topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))


In [122]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target_cls) in enumerate(val_loader):
        
        target = torch.zeros(batch_size, 200)
        for ind, cls in enumerate(target_cls):
            target[ind, cls-1] = 1
        target = target.float()
        target = target.cuda(async=True)
        
        input = input.float()
        input = input.cuda(async=True)
        
        input_var = torch.autograd.Variable(input, volatile=True).cuda()
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, torch.LongTensor(target_cls).cuda(), topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))

    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

In [5]:
classifier_size = 512 * 2 * 2

model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}

class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(classifier_size, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    'LITTLE': [16, 'M', 32, 'M', 64, 64, 'M', 128, 128, 'M', 128, 128, 'M']
}


def vgg11(pretrained=False, **kwargs):
    """VGG 11-layer model (configuration "A")
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfg['A']), **kwargs)
    if pretrained:
        model.load_state_dict( model_zoo.load_url(model_urls['vgg11']))
    return model


def vgg11_bn_little(pretrained=False, **kwargs):
    """VGG 11-layer model (configuration "A") with batch normalization
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfg['A'], batch_norm=True), **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['vgg11_bn']))
    return model

In [110]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = init_lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [3]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')